<center><h1>Using Grover's Algorithm on Knights</h1></center>

Knights are renowned as the most elusive of the chess pieces, moving in "L" shapes rather than straight lines or diagonals. Here we use Grover's Algorithm to solve for the knight positions in which two knights attack each other on a $4 \times 4$ board.

<img width="420" alt="Knights Photo" src="https://user-images.githubusercontent.com/42923017/121147835-3d184f00-c80f-11eb-8cc5-a9710f9955fc.png">

### Setup

To help formalize the problem, we represent the board using a $4 \times 4$ grid on the $x$-$y$ plane, with each knight's position represented by a pair of 2-bit coordinates $([x_2, x_1], [y_2, y_1])$. We use 2 bits for both the $x$ and $y$ coordinates since the 4 rows or columns can be encoded in 2 bits from $00$ to $11$.

Then, the positions of the knights can be written as

$$
([\text{knight1_}x_2, \text{knight1_}x_1],\space [\text{knight1_}y_2, \text{knight1_}y_1]),\space ([\text{knight2_}x_2, \text{knight2_}x_1],\space [\text{knight2_}y_2, \text{knight2_}y_1]),
$$

as demonstrated below:

<img width="480" alt="Knights_151_labels" src="https://user-images.githubusercontent.com/42923017/121154321-e0b82e00-c814-11eb-83f7-ea2a682db6f1.png">

The positions can also be written as an 8-qubit array of the form

$$
[\text{knight1_}x_2, \text{knight1_}x_1, \text{knight1_}y_2, \text{knight1_}y_1, \text{knight2_}x_2, \text{knight2_}x_1, \text{knight2_}y_2, \text{knight2_}y_1],
$$

which we use as the register for our marking oracle.

### Implementation of Oracle

To check that two knights on a 4x4 board attack each other, we check that their positions form an "L" shape, i.e. their $x$ coordinates differ by 1 and their $y$ coordinates differ by 2, or vice versa. Without loss of generality, we  only elaborate the former case. 


Let us first consider checking if two 2-bit coordinates differ by 1.

The pairs of 2-bit coordinates which differ by 1 are as follows:
\begin{align*}
    00 &\iff 01\\
    01 &\iff 10\\
    10 &\iff 11
\end{align*}

In each pair, the end bits are distinct, so we use `CX` with the end bits as controls on an ancilla qubit `xleap01` (initialized to `false`) to track this bit distinction. `xleap01` becomes `true` when the end bits are distinct. However, there is one particular edge case: $00$ and $11$ have distinct end bits but do not differ by 1. To account for this, we use `ControlledOnBitString` with `[false, false, true, true]` and `[true, true, false, false]` as controls on `xleap01` to revert it back to `false` if the pair of 2-bit coordinates is $0011$ or $1100$.


Now we consider checking if two 2-bit coordinates differ by 2.

The pairs of 2-bit coordinates which differ by 2 are as follows:
\begin{align*}
    00 &\iff 10\\
    01 &\iff 11
\end{align*}

In each pair, the the end bits are identical while start bits are distinct, so we use `CX` with the end and start bits as controls on ancilla qubits `yleap10_same` and `yleap10_diff` (both initialized to `false`) to track these bit relations. `yleap10_same` remains `false` when the end bits are identical and `yleap10_same` becomes `true` when the start bits are distinct.

Combining both difference checks, we use `ControlledOnBitString` with `[true, false, true]` as controls on the `target` qubit.

Below, we implement this logic in `KnightsMarkingOracle`, where we consider both cases of "L" shapes by doubling the statements:

In [1]:
// Import necessary packages

open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;


In [2]:
operation KnightsMarkingOracle(
    coordsRegister: Qubit[],
    target: Qubit
): Unit is Adj + Ctl {
    // knight 1/2 denotes 1st or 2nd knight
    // x/y 2/1 denotes start/end of the two bits in the x/y coordinate
    let (knight1_x2, knight1_x1, knight1_y2, knight1_y1, knight2_x2, knight2_x1, knight2_y2, knight2_y1) = 
    (coordsRegister[0], coordsRegister[1], coordsRegister[2], coordsRegister[3], 
     coordsRegister[4], coordsRegister[5], coordsRegister[6], coordsRegister[7]);

    // xleap01 ancilla qubit denotes if x's differ by 1
    // yleap10_same ancilla qubit denotes if y ends remain the same
    // yleap10_diff ancilla qubit denotes if y starts differ by 2
    use xleap01 = Qubit();
    use yleap10_same = Qubit();
    use yleap10_diff = Qubit();
    
    // yleap01 ancilla qubit denotes if y's differ by 1
    // xleap10_same ancilla qubit denotes if x ends remain the same
    // xleap10_diff ancilla qubit denotes if x starts differ by 2
    use yleap01 = Qubit();
    use xleap10_same = Qubit();
    use xleap10_diff = Qubit();
     
    within {
        //// knight leap: 1 over, 2 up/down
        
        // end x coords are diff if x's differ by 1
        CX(knight1_x1, xleap01);
        CX(knight2_x1, xleap01);
        
        
        // handle 0011 and 1100 edge cases
        ControlledOnBitString([false, false, true, true], X)([knight1_x2, knight1_x1, knight2_x2, knight2_x1], xleap01);
        ControlledOnBitString([true, true, false, false], X)([knight1_x2, knight1_x1, knight2_x2, knight2_x1], xleap01);
        
        // end y coords are same and start y coords are diff if y's differ by 2
        CX(knight1_y1, yleap10_same);
        CX(knight2_y1, yleap10_same);
        
        CX(knight1_y2, yleap10_diff);
        CX(knight2_y2, yleap10_diff);
        
        //// knight leap: 1 up/down, 2 over
        
        // end y coords are diff if y's differ by 1
        CX(knight1_y1, yleap01);
        CX(knight2_y1, yleap01);
        
        // handle 0011 and 1100 edge cases
        ControlledOnBitString([false, false, true, true], X)([knight1_y2, knight1_y1, knight2_y2, knight2_y1], yleap01);
        ControlledOnBitString([true, true, false, false], X)([knight1_y2, knight1_y1, knight2_y2, knight2_y1], yleap01);
        
        // end x coords are same and start x coords are diff if x's differ by 2
        CX(knight1_x1, xleap10_same);
        CX(knight2_x1, xleap10_same);
        
        CX(knight1_x2, xleap10_diff);
        CX(knight2_x2, xleap10_diff);
    }
    apply {
        // check true for leap01 because false -> true when a difference is detected
        // check false for leap10_same because false -> false when both controls are the same
        // check true for leap10_diff because false -> true when a difference is detected
        ControlledOnBitString([true, false, true], X)([xleap01, yleap10_same, yleap10_diff], target);
        ControlledOnBitString([true, false, true], X)([yleap01, xleap10_same, xleap10_diff], target);
    }
}

KnightsMarkingOracle

We may also test our knights marking oracle by inputting an integer that is converted to a register of 8 qubits (in BigEndian order) for the oracle. For example, our test case of `inputNum=151` is converted in the following manner:

$$
151 = \textbf{1} \cdot 2^7 + \textbf{0} \cdot 2^6 + \textbf{0} \cdot 2^5 + \textbf{1} \cdot 2^4 + \textbf{0} \cdot 2^3 + \textbf{1} \cdot 2^2 + \textbf{1} \cdot 2^1 + \textbf{1} \cdot 2^0 \implies [|1\rangle, |0\rangle, |0\rangle, |1\rangle, |0\rangle, |1\rangle, |1\rangle, |1\rangle].
$$

Then, the target result is displayed through `DumpRegister`, where $|1\rangle$ and $|0\rangle$ represent attacking and non-attacking knights, respectively. In the provided test case, we have attacking knights since the first knight at $([1, 0], [0, 1])$ attacks the second knight at $([0, 1], [1, 1])$, and vice versa. We depict these attacking knights with our function `printBoard` when we simulate our test case.

It is not required, but the user may try any `inputNum` between `0` and `255`.


In [3]:
function printBoard(inputNum : Int) : Unit {
    Message("");
    Message("    00      01      10      11");
    let (x1, y1, x2, y2) = (inputNum / 64, ModI(inputNum, 64) / 16, ModI(inputNum, 16) / 4, ModI(inputNum, 4));
    let (L, W) = (4, 8);
    for r in 0..(4 * L) {
        mutable row = "";
        for c in 0..(4 * W) {
            if ModI(r, L) == 0 {
                if ModI(c, W) > 0 {
                    set row += "-";
                }
                else {
                    set row += "+";
                }
            }
            else {
                if ModI(c, W) == 0 {
                    set row += "|";
                }
                else {
                    if (r / L == 3 - y1 and c / W == x1) {
                        set row += "1";
                    }
                    elif (r / L == 3 - y2 and c / W == x2) {
                        set row += "2";
                    }
                    else {
                        set row += " ";
                    }
                }
            }
        }
        
        if r == 2 {
            set row += " 11";
        }
        elif r == 6 {
            set row += " 10";
        }
        elif r == 10 {
            set row += " 01";
        }
        elif r == 14 {
            set row += " 00";
        }
        
        Message($"{row}");
    }
    Message("");
}

printBoard

In [4]:
operation testKnightsOracle (inputNum : Int) : Unit {
    // current test case of 151; knight 1: ([1, 0], [0, 1]), knight 2: ([0, 1], [1, 1])
    printBoard(inputNum);
    let inputBits = IntAsBoolArray(inputNum, 8);
    use inputRegister = Qubit[8];
    for i in 0..7 {
        if inputBits[i] {
            X(inputRegister[7 - i]);
        }
    }
    use target = Qubit();
    KnightsMarkingOracle(inputRegister, target);
    Message("target:");
    DumpRegister((), [target]);
    ResetAll(inputRegister);
    Reset(target);
}

testKnightsOracle

In [5]:
%simulate testKnightsOracle inputNum=151


    00      01      10      11
+-------+-------+-------+-------+
|       |2222222|       |       |
|       |2222222|       |       | 11
|       |2222222|       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 10
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |1111111|       |
|       |       |1111111|       | 01
|       |       |1111111|       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 00
|       |       |       |       |
+-------+-------+-------+-------+

target:


|0⟩	0 + 0𝑖
|1⟩	1 + 0𝑖

()

`target` being in the $|1\rangle$ state is consistent with the knights attacking each other.

### Using Grover's Algorithm

Now, we use the oracle implemented in the previous part as part of a Grover search. The end result should yield high amplitudes for basis states representing two attacking knights, and near-zero amplitudes otherwise. Therefore, when we perform a measurement in the computational basis, we should expect to obtain one of the states that represents two attacking knights.

In the `OracleConverter` operation below, we convert the previously implemented marking oracle into a phase oracle. This allows us to flip the phases of the marked states so that `GroverAlgorithmLoop` afterwards can repeatedly perform mean inversion to amplify their probability amplitudes.

In [6]:
// Oracle Converter, GroverAlgorithmLoop, and GroversAlgorithm implemented in Graph Coloring Katas.

// Convert above marking oracle to a phase oracle, using phase-kickback

operation OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    use target = Qubit();
    within {
        // Put the target qubit in the |-⟩ state
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle
        markingOracle(register, target);
    }
}

operation GroverAlgorithmLoop (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[], iterations : Int) : Unit is Adj {
    // Convert the marking oracle in a phase oracle
    let phaseOracle = OracleConverter(markingOracle, _);
    // Prepare an equal superposition of all basis states
    ApplyToEachA(H, register);
    // Apply Grover iterations
    for _ in 1..iterations {
        // Apply phase oracle
        phaseOracle(register);
        // Apply "reflection about the mean"
        within {
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } apply {
            (Controlled Z)(Most(register), Tail(register));
        }
    }
}

OracleConverter, GroverAlgorithmLoop

In `GroversAlgorithm`, we run `GroverAlgorithmLoop` for a certain number of iterations (theoretical calculation given below with $M = 48$ and $N = 256$) so that the measurement of a marked output state returns `One`, in which case we have found our positions of two attacking knights. Then, `MeasureConfig` measures the configuration to extract the Boolean array of knight positions.

$M = 48$ : Given an "L" shape, there are 6 ways to place two knights on the board that attack each other in that "L" shape. Then, there are 4 possible orientations of the "L" shape and 2 possible orderings of the knights (first or second), making for a total of $6\cdot 4 \cdot 2 = 48$ total unique positions of attacking knights.

$N = 256$ : Since there are 8 qubits and 2 states for each qubit ($|0\rangle$ or $|1\rangle$), there are $2^8 = 256$ total positions of knights. 

$$k = \frac{\pi}{4\arcsin{\sqrt{\frac{M}{N}}}} - \frac{1}{2} 
= \frac{\pi}{4\arcsin{\sqrt{\frac{48}{256}}}} - \frac{1}{2} 
\approx 1.254$$

In [7]:
operation GroversAlgorithm (oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    mutable config = new Bool[8];
    use (register, output) = (Qubit[8], Qubit());
    mutable correct = false;
    mutable iterations = 1;
    repeat {
        // Message($"Trying iteration {iterations}");
        GroverAlgorithmLoop(oracle, register, iterations);
        let temp = MultiM(register);
        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            set config = MeasureConfig(register);
        }
        ResetAll(register);
    }
    until (correct or iterations > 10)
    fixup {
        set iterations += 1;
    }
    if (not correct) {
        fail "The two knights do not attack each other.";
    }
    return config;
}

operation MeasureConfig (register : Qubit[]) : Bool[] {
    let measurements = MultiM(register);
    // Reversed for BigEndian order
    return Reversed(ResultArrayAsBoolArray(measurements));
}

GroversAlgorithm, MeasureConfig

The above algorithm will only return 1 set of attacking knights, so we write an operation `runGroverKnightsUnique` to run the algorithm multiple times and generate all unique positions of attacking knights, displayed using `printBoard`:

In [8]:
function b2i(b : Bool) : Int {
    return b ? 1 | 0;
}

b2i

In [9]:
function tupleEquality (tuple1: (Int, Int, Int, Int, Int, Int, Int, Int), tuple2: (Int, Int, Int, Int, Int, Int, Int, Int)) : Bool {
    let (a1, b1, c1, d1, e1, f1, g1, h1) = tuple1;
    let (a2, b2, c2, d2, e2, f2, g2, h2) = tuple2;
    return a1 == a2 and b1 == b2 and c1 == c2 and d1 == d2 and e1 == e2 and f1 == f2 and g1 == g2 and h1 == h2;
}

tupleEquality

In [10]:
operation runGroverKnightsUnique(num_iters : Int) : (Int, Int, Int, Int, Int, Int, Int, Int)[] {
    mutable runs = new (Int, Int, Int, Int, Int, Int, Int, Int)[num_iters];
    mutable r = 0;
    for _ in 1..num_iters {
        let attackingKnights = GroversAlgorithm(KnightsMarkingOracle);
        let uniqueCand = 
        (b2i(attackingKnights[0]), b2i(attackingKnights[1]), b2i(attackingKnights[2]), b2i(attackingKnights[3]), 
         b2i(attackingKnights[4]), b2i(attackingKnights[5]), b2i(attackingKnights[6]), b2i(attackingKnights[7]));
        let isEqual = tupleEquality(uniqueCand, _);
        // add uniqueCand to runs if it is unique
        if IndexOf(isEqual, runs) < 0 {
            printBoard(BoolArrayAsInt(attackingKnights));
            set runs w/= r <- uniqueCand;
            set r += 1;
        }
    }
    Message($"Total Number of Valid Configurations:\t{r}");
    return runs[0..(r - 1)];
}

runGroverKnightsUnique

In [11]:
%simulate runGroverKnightsUnique num_iters=256


    00      01      10      11
+-------+-------+-------+-------+
|       |       |1111111|       |
|       |       |1111111|       | 11
|       |       |1111111|       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 10
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |       |2222222|
|       |       |       |2222222| 01
|       |       |       |2222222|
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 00
|       |       |       |       |
+-------+-------+-------+-------+


    00      01      10      11
+-------+-------+-------+-------+
|2222222|       |       |       |
|2222222|       |       |       | 11
|2222222|       |       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 10
|       |       |       |       |
+-------+-------+-------+-------+
|       |1111111|       |       |

|1111111|       |       |       | 11
|1111111|       |       |       |
+-------+-------+-------+-------+
|       |       |2222222|       |
|       |       |2222222|       | 10
|       |       |2222222|       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 01
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 00
|       |       |       |       |
+-------+-------+-------+-------+


    00      01      10      11
+-------+-------+-------+-------+
|       |2222222|       |       |
|       |2222222|       |       | 11
|       |2222222|       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 10
|       |       |       |       |
+-------+-------+-------+-------+
|1111111|       |       |       |
|1111111|       |       |       | 01
|1111111|       |       |       |
+-------+-------+-------+---

|1111111|       |       |       |
|1111111|       |       |       | 10
|1111111|       |       |       |
+-------+-------+-------+-------+
|       |       |2222222|       |
|       |       |2222222|       | 01
|       |       |2222222|       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 00
|       |       |       |       |
+-------+-------+-------+-------+


    00      01      10      11
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 11
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |1111111|       |
|       |       |1111111|       | 10
|       |       |1111111|       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 01
|       |       |       |       |
+-------+-------+-------+-------+
|       |2222222|       |       |
|       |2222222|       |       | 00
|       |2222222|       |   

+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 01
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |1111111|       |
|       |       |1111111|       | 00
|       |       |1111111|       |
+-------+-------+-------+-------+


    00      01      10      11
+-------+-------+-------+-------+
|       |       |       |1111111|
|       |       |       |1111111| 11
|       |       |       |1111111|
+-------+-------+-------+-------+
|       |2222222|       |       |
|       |2222222|       |       | 10
|       |2222222|       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 01
|       |       |       |       |
+-------+-------+-------+-------+
|       |       |       |       |
|       |       |       |       | 00
|       |       |       |       |
+-------+-------+-------+-------+


    00      01      10      11
+-------+-------+-------+-------

(1, 0, 1, 1, 1, 1, 0, 1), (1, 1, 0, 0, 1, 0, 1, 0), (0, 0, 1, 1, 0, 1, 0, 1), (0, 1, 0, 1, 0, 0, 1, 1), (1, 0, 1, 1, 0, 0, 1, 0), (1, 1, 0, 0, 0, 1, 0, 1), (0, 1, 0, 0, 0, 0, 1, 0), (1, 0, 0, 0, 1, 1, 1, 0), (1, 1, 1, 0, 1, 0, 0, 1), (1, 1, 0, 1, 1, 0, 1, 0), (1, 1, 0, 1, 0, 1, 0, 0), (0, 0, 1, 0, 0, 1, 0, 0), (1, 0, 1, 0, 0, 1, 1, 1), (0, 1, 0, 1, 1, 1, 0, 0), (1, 1, 1, 0, 1, 0, 0, 0), (1, 0, 1, 1, 0, 1, 1, 0), (0, 1, 0, 1, 0, 0, 1, 0), (0, 1, 1, 1, 1, 0, 1, 0), (0, 0, 0, 1, 0, 1, 1, 1), (0, 0, 1, 0, 1, 0, 1, 1), (1, 0, 0, 0, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0, 0, 1), (1, 1, 1, 1, 0, 1, 1, 0), (0, 1, 1, 0, 0, 0, 0, 1), (1, 0, 0, 1, 1, 1, 1, 0), (1, 0, 0, 1, 1, 1, 1, 1), (1, 0, 0, 1, 0, 1, 0, 0), (0, 0, 1, 0, 0, 1, 0, 1), (1, 0, 1, 0, 1, 1, 0, 0), (0, 0, 0, 1, 1, 0, 0, 0), (1, 1, 1, 0, 0, 1, 1, 1), (1, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 1, 0), (0, 1, 1, 1, 1, 1, 1, 0), (1, 0, 1, 0, 1, 1, 0, 1), (1, 1, 0, 1, 1, 0, 1, 1), (0, 0, 0, 1, 0, 1, 1, 0), (0, 1, 0, 0, 1, 0, 0, 1), (0, 1, 0, 0, 1, 1, 0, 1), (0, 1, 1, 1, 0, 0, 0, 1), (0, 1, 1, 0, 1, 1, 1, 1), (1, 1, 1, 1, 1, 0, 0, 1), (1, 0, 0, 1, 0, 0, 0, 0), (0, 1, 0, 1, 1, 0, 0, 0), (1, 0, 1, 0, 0, 0, 1, 1), (0, 0, 1, 1, 1, 0, 1, 0), (0, 1, 1, 0, 1, 0, 1, 1), (0, 1, 1, 0, 0, 0, 0, 0)

As expected, there are $48$ total unique positions of attacking knights.

Possible extensions: more knights, reachable squares in k moves, max num of non-attacking knights ( floor( (n^2 + 1) / 2 ) )
https://www.geeksforgeeks.org/place-k-knights-such-that-they-do-not-attack-each-other/
https://www.geeksforgeeks.org/maximum-non-attacking-knights-that-can-be-placed-on-an-nm-chessboard/